In [24]:
## CELL 1 to execute ##
# install nltk
# install pandas
# install numpy
# import nltk
#import glob
# install num2words
# nltk.download('stopwords')
# nltk.download('punkt')

import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

ERROR! Session/line number was not unique in database. History logging moved to new session 61


In [25]:
## CELL 2 to execute ##
pip install num2words

In [0]:
## CELL 3 to execute ##
#import all necessities 

from nltk.stem import PorterStemmer 
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize 
import numpy as np
import glob
import os
from num2words import num2words
from nltk.corpus import stopwords 
import pickle
import math
import operator
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt 

ps = PorterStemmer() 
tokenizer = RegexpTokenizer(r'\w+')


In [5]:
## CELL 4 to execute ##
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
### CELL Inverted List ###
### PLEASE NOTE THAT Inv_list is "PICKLED" (myInverted_List_IR4.pickle) in order to save time of creating inverted list again and again ###
###Please avoid executing this code.Inverted list is alreadu PICKLED ####
#### Inverted List (Inv_list) with tf values ####
#### Final inverted list (INV) with TF-IDF value is created in later lines of code ###

stop_words = set(stopwords.words('english')) 
Inv_list = {} 
cat_list=["comp.graphics","rec.sport.hockey","sci.med","sci.space","talk.politics.misc"]
for group in cat_list:
  for filename in glob.glob("drive/My Drive/IR4/"+group+"/*"):
    
    length=len(filename)
    j=0
    for k in range(length-1,-1,-1):
      if filename[k]=='/':
        j=k
        break
    print(filename[j+1:len(filename)])
    if not os.path.isdir(filename):
      text = open(filename,"r+",errors='ignore') 
      var=(text.read())
      var=var.replace(',','')
      words_tokens=tokenizer.tokenize(var)
      words = [w for w in words_tokens if not w in stop_words] 
 

      
      Words=[]
      for w in words:
        if w.isdigit():
  
          Words.append(ps.stem(num2words(w)))
        else:
          Words.append(ps.stem(w))

      for word in Words:
        
        if word not in Inv_list.keys():
          List=[tuple([filename[j+1:len(filename)],1])]
          Inv_list.update({word:List})
        else:
          val=Inv_list.get(word)
          flag=0
          i=0
          c=0
          for v in val:
            if v[0] ==filename[j+1:len(filename)]:

              flag=1
              c=i
              break
            i=i+1

          if flag==0:
            val.append(tuple([filename[j+1:len(filename)],1]))
          if flag==1:
            temp=val[c]
            val.pop(c)
            t=tuple([temp[0],temp[1]+1])
            val.append(t)




In [0]:
### Code to Create Pickle file for Inverted List ###
pickle_out = open("drive/My Drive/myInverted_List_IR4.pickle","wb")
pickle.dump(Inv_list, pickle_out)
pickle_out.close()

In [0]:
## CELL 5 to execute ##
### Code to extract inverted List from Pickled file ###

pickle_in = open("drive/My Drive/myInverted_List_IR4.pickle","rb")
Inv_list= pickle.load(pickle_in)


In [28]:
## CELL 6 to execute ##
### Code to calculate total number of documents in the corpus ### 

Num=0
cat_list=["comp.graphics","rec.sport.hockey","sci.med","sci.space","talk.politics.misc"]
for group in cat_list:
  for filename in glob.glob("drive/My Drive/IR4/"+group+"/*"):
    Num=Num+1
print(Num)

5000


In [0]:
## CELL 7 to execute ##
### Final inverted list (Inv) with TF-IDF values ###

Inv={}
Inv_final={}
for key in Inv_list.keys():
  df=len(Inv_list.get(key))
  df=math.log((Num/df),10)
  val=Inv_list.get(key)
  List=[]
  for item in val:
    tf_idf=(1+math.log(item[1],10))*df
    List.append(tuple([item[0],tf_idf]))
  Inv_final.update({key:List})
Inv_list=Inv_final 

def last(n):
  return n[1]   

for key in Inv_list.keys():
  temp=Inv_list.get(key)
  temp=sorted(temp, key = last,reverse=True)
  Inv.update({key:temp})

In [0]:

## Code to create document vectors (Docs_vec) for all 5k documents in the corpus ###
## PLEASE NOTE THAT Docs_vec is PICKLED (Docs_Vec_IR4.pickle) in order to save time of creating Document vectors again and again ###
### Avoid executing this code.Use PICKELD File
def create_doc_vec():

  Docs_vec={}
  cat_list=["comp.graphics","rec.sport.hockey","sci.med","sci.space","talk.politics.misc"]
  for group in cat_list:
    for filename in glob.glob("drive/My Drive/IR4/"+group+"/*"):
        #D_vec = [0 for i in range(0,len(Inv_list))]
      print("filename",filename[j+1:len(filename)])
      D_vec=[]
      length=len(filename)
      j=0
      for k in range(length-1,-1,-1):
        if filename[k]=='/':
          j=k
          break
      
      for key in Inv.keys():
        flag11=0
        list1=Inv.get(key)
        for dt in list1:
          if dt[0]==filename[j+1:len(filename)]:
            D_vec.append(dt[1])
            flag11=1
        if flag11==0:
          D_vec.append(0)
      print(len(D_vec))
      Docs_vec.update({filename[j+1:len(filename)]:D_vec})
  return Docs_vec  




In [0]:
## Code to dump all the document vectors created in the pickle file called "Docs_Vec_IR4.pickle"

Docs_vec=create_doc_vec()
pickle_out = open("Docs_Vec_IR4.pickle","wb")
pickle.dump(Docs_vec, pickle_out)
pickle_out.close()

In [0]:
## CELL 8 to execute ##
##Code to load all the document vectors from "Docs_Vec_IR4.pickle"  ###

pickle_in = open("drive/My Drive/Docs_Vec_IR4.pickle","rb")
Docs_vec= pickle.load(pickle_in)

In [0]:
## CELL 9 to execute ##
## Cosine(Q_vec,k) is used to calculate cosine similarity between query vectors and document vectors ###
## This function takes 2 paramers Q_vec ::: Query Vector and k ::: Number of results user wants to display ###
### This function returns Ranked top k documents with highest cosine similarity with the query vector ###


def cosine(Q_vec,k):  
  mod_q=0
  for item in Q_vec:
    mod_q=mod_q+ item**2
  mod_q=math.sqrt(mod_q)
  Result=[]
  for doc in Docs_vec.keys():
    D_vec=Docs_vec.get(doc)
    mod_d=0
    for item in D_vec:
      mod_d=mod_d+ item**2
    mod_d=math.sqrt(mod_d)
    Res=0
    for i in range(0,len(D_vec)):
      Res=Res+(D_vec[i]*Q_vec[i])
    Res=Res/(mod_d*mod_q)
    Result.append(tuple([doc,Res,D_vec]))


  Result.sort(key = operator.itemgetter(1),reverse=True)
  Result1=[]


  for i in range(0,k):
    Result1.append(Result[i])
  return Result1
 



In [0]:
## CELL 10 to execute ##
### graph() function is used to plot 2D TSNE graph of the vectors to demonstrate known relevant , non relevan and the query vector.

def graph(R_vecs,NR_vecs,Q_vec):
    tsne = TSNE(n_components=2, random_state=0,verbose=0, perplexity=40)

    vector=[]
    labels=[]
    color=[]
    for item in R_vecs:
        vector.append(item)
        color.append('m')
    for item in NR_vecs:
        vector.append(item)
        color.append('y')
    
    vector.append(Q_vec)
    color.append('k')
    transformed_data = tsne.fit_transform(np.array(vector))
    k = np.array(transformed_data)
    t = ("Relevant", "Non Relevant", "Query")
    plt.scatter(k[:, 0], k[:, 1],c=color,label="magenta - Relevant,Yellow - Non Relevant,black - Query Vector ")
    plt.title("Rocchio Algorithm")
    plt.legend()
    plt.grid(True)
    plt.show()
    return

In [0]:
## CELL 11 to execute ##
### calc_PR() is used to plot precision - recall curve for each iteration of relevance feedback ###

def calc_PR(rel_array):
  precision=[]
  recall=[]
  i=0
  rel=0
  total=1000

  for item in rel_array:
    val=item
    rel=rel+val
    precision.append(rel/(i+1))
    recall.append(rel/total)
    i=i+1

  print("precision: ", precision)
  print("recall: ", recall)  


  
  plt.plot(recall,precision) 


  plt.ylabel('precision') 

  plt.xlabel('recall') 
  plt.title('P-R plot') 
  
  plt.show() 


In [0]:
## CELL 12 to execute ##
### Function to calculate Average precision for the set of queries ###

def AP(rel_array):
  precision=[]
  i=0
  rel=0
  

  for item in rel_array:
    val=item
    rel=rel+val
    precision.append(rel/(i+1))
    i=i+1

  ap=0
  count=0
  j=0
  for item in rel_array:
    if item==1:
      count=count+1
      ap=ap+precision[j]  
    j=j+1

  ap=ap/count
  return ap


In [0]:
## CELL 13 to execute ##
### Function to caculate mean average precision MAP ###

def MAP(res_arr):
  length=len(res_arr[0])
  ct=len(res_arr)
  for i in range(0,length):
    sum=0
    for j in range(0,ct):
      sum=sum+res_arr[j][i]
    print("MAP for ",i+1," iteration is: ",sum/ct)




In [0]:
## CELL 14 to execute ##
#### Heart string of this code file ###
#### this piece of code performs the various tasks with the help of other function ### 
try :

  q=int(input("how many queries you wish you input"))
  LIL = [[] for i in range(q)]

  file_Dict={1:"comp.graphics", 2:"rec.sport.hockey", 3:"sci.med", 4:"sci.space", 5:"talk.politics.misc"}
  z=0
  while q>0:
    q=q-1
    
    query=input("Enter your query")
    k=input("Enter number of search results you want")
    k=int(k)
    print("Enter ground truth folder required to plot PR curve in part e: ")
    print("1:comp.graphics  ", "2:rec.sport.hockey  ", "3:sci.med  ", "4:sci.space  ", "5:talk.politics.misc")
    print("Enter a value from 1-5")
    choice=int(input())


    query=query.replace(',','')

    query_stem_original=[]

    words=tokenizer.tokenize(query)



    for w in words:
      if w.isdigit():
        query_stem_original.append(ps.stem(num2words(w)))
      else:  
        query_stem_original.append(ps.stem(w))  

    query_stem=sorted(set(query_stem_original))

    Q_vec=[]

    for key in Inv.keys():
      if key in query_stem:
        tf=0
        for t in query_stem_original:
          if t==key:
            tf=tf+1
        df=len(Inv_list.get(key))
        df=math.log((Num/df),10)
        tf_idf=(1+math.log(1+tf,10))*df
        Q_vec.append(tf_idf)
      else:
        Q_vec.append(0)


    Rel_Array=[]
    iter=3
    rty.clear()
    while(iter>0):
      iter=iter-1
      feed_back=[0 for i in range(0,k)]
      Result=cosine(Q_vec,k)
      
      for item in Result:
        flag_rel=False
        for filename in glob.glob("drive/My Drive/IR4/"+file_Dict.get(choice)+"/*"):
          length=len(filename)
          j=0
          for k1 in range(length-1,-1,-1):
            if filename[k1]=='/':
              j=k1
              break
          


          if item[0]==filename[j+1:len(filename)]:
            flag_rel=True
        
        if flag_rel==True:
          Rel_Array.append(1)
        else:
          Rel_Array.append(0)

      calc_PR(Rel_Array)
      ll=AP(Rel_Array)
      LIL[z].append(ll)
      
      for i in range(0,len(Result)):
        if Result[i][0] in rty:
          print(i,Result[i][0],"*")  
        else:
          print(i,Result[i][0])

      nos=input("Enter relevant docs list.Please use space to sepate two doc IDs ")
      
      nos=nos.split()
      rel=[]
      for item in nos:
        rel.append(int(item))
      non_rel=[]
      for item in range(0,k):
        if item not in rel:
          non_rel.append(item)
      R_vecs=[]
      NR_vecs=[]    
      R=np.array([0 for i in range(0,len(Q_vec))])
      N=np.array([0 for i in range(0,len(Q_vec))])
      rty=[]
      for item in rel:
        rty.append(Result[item][0])
        R_vecs.append(np.array(Result[item][2]))
        R=np.add(np.array(Result[item][2]),R)
        
      for item in non_rel:
        NR_vecs.append(np.array(Result[item][2]))
        N=np.add(np.array(Result[item][2]),N)

      N=(N/len(rel))
      R=(R/len(non_rel))
      Q_vec=Q_vec + 0.7 * R - 0.25*N  
      
      for item in Q_vec:
        if item<0:
          item=0
      print(Q_vec)
      print("Average precision of the query for this iteration is :",ll)
      print("Please hold tight!!! Mean average precision (MAP) will be displayed after all the feedback iterations of every query")
      
      graph(R_vecs,NR_vecs,Q_vec)
    z=z+1

  MAP(LIL)  
except :
  print("Some exception occured")

how many queries you wish you input1
Enter your queryPretty good opinions on biochemistry machines
Enter number of search results you want100
Enter ground truth folder required to plot PR curve in part e: 
1:comp.graphics   2:rec.sport.hockey   3:sci.med   4:sci.space   5:talk.politics.misc
Enter a value from 1-5
3


In [42]:
print(rty)

['54516', '53935', '54273', '59400', '59411', '38555', '58959', '62410', '178963', '61190']
